In [1]:
N = (7 + 4 ) % 5 + 1
print("Variant " + str(N))

Variant 2


In [3]:
import pandas as pd

# Завантаження даних з файлу kr1.xlsx в датафрейм
file_path = 'lab6.xlsx'  # замініть на шлях до вашого файлу
df = pd.read_excel(file_path)

# Виведення перших кількох рядків, щоб переконатись у правильності завантаження
print(df.head())

   N         file name image size glasses color  line width
0  1  emma-watson2.jpg    300x300           red           2
1  2   emma-watson.jpg    400x400          blue           3
2  3         draco.jpg    700x700          cian           4
3  4    longbottom.jpg    600x600       magenta           5
4  5    ron_wesley.jpg    500x500        yellow           6


In [39]:
import cv2
from PIL import Image, ImageDraw

# Загрузка каскадов для распознавания лиц и глаз с использованием каскада для очков
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye_tree_eyeglasses.xml')

# Функция для добавления очков на изображение
def add_glasses(image_path, output_path, glasses_color, line_width):
    # Чтение изображения через OpenCV
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Определение лиц на изображении
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=4)
    print(f'Количество найденных лиц: {len(faces)}')

    if len(faces) == 0:
        print("Лицо не найдено")
        return
    
    # Открываем изображение через PIL для рисования
    pil_img = Image.open(image_path)
    draw = ImageDraw.Draw(pil_img)

    # Проходим по всем лицам
    for (x, y, w, h) in faces:
        print(f"Найдено лицо с координатами: x={x}, y={y}, ширина={w}, высота={h}")
        
        # Область лица
        face_roi = gray[y:y + h, x:x + w]
        roi_color = img[y:y+h, x:x+w]

        # Определение глаз в области лица с использованием нового каскада
        eyes = eye_cascade.detectMultiScale(face_roi, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        print(f'Количество найденных глаз: {len(eyes)}')

        if len(eyes) < 2:
            print(f"Найдено глаз: {len(eyes)}. Недостаточно глаз для добавления очков.")
            continue
        
        # Отбираем два глаза и проверяем, что они находятся на одной линии
        eyes = sorted(eyes, key=lambda ex: ex[0])[:2]
        
        # Проверяем, что глаза находятся примерно на одной высоте (по вертикальной координате)
        if abs(eyes[0][1] - eyes[1][1]) > 20:  # Порог 20 пикселей
            print(f"Глаза не на одной линии: разница в высоте {abs(eyes[0][1] - eyes[1][1])} пикселей, пропуск изображения.")
            continue  # Пропускаем изображение
        
        # Координаты глаз
        left_eye = (x + eyes[0][0] + eyes[0][2] // 2, y + eyes[0][1] + eyes[0][3] // 2)
        right_eye = (x + eyes[1][0] + eyes[1][2] // 2, y + eyes[1][1] + eyes[1][3] // 2)

        print(f"Координаты левого глаза: {left_eye}, правого глаза: {right_eye}")
        
        # Радиус очков
        radius = (right_eye[0] - left_eye[0]) // 3
        
        # Рисуем очки
        draw.ellipse((left_eye[0] - radius, left_eye[1] - radius, left_eye[0] + radius, left_eye[1] + radius),
                     outline=glasses_color, width=line_width)
        draw.ellipse((right_eye[0] - radius, right_eye[1] - radius, right_eye[0] + radius, right_eye[1] + radius),
                     outline=glasses_color, width=line_width)
        
        # Рисуем переносицу
        draw.line((left_eye[0] + radius, left_eye[1], right_eye[0] - radius, right_eye[1]), 
                  fill=glasses_color, width=line_width)

        # Рисуем заушники
        draw.line((left_eye[0] - radius, left_eye[1], left_eye[0] - radius - 50, left_eye[1]), 
                  fill=glasses_color, width=line_width)
        draw.line((right_eye[0] + radius, right_eye[1], right_eye[0] + radius + 50, right_eye[1]), 
                  fill=glasses_color, width=line_width)

    # Сохраняем и показываем результат
    pil_img.save(output_path)
    pil_img.show()



In [40]:
add_glasses('emma-watson.jpg', 'output_with_glasses.jpg', 'red', 5)

Количество найденных лиц: 1
Найдено лицо с координатами: x=90, y=106, ширина=226, высота=226
Количество найденных глаз: 2
Координаты левого глаза: (166, 190), правого глаза: (244, 196)


In [42]:
add_glasses('draco.jpg', 'output_with_glasses.jpg', 'red', 5)

Количество найденных лиц: 1
Найдено лицо с координатами: x=178, y=185, ширина=351, высота=351
Количество найденных глаз: 2
Координаты левого глаза: (289, 324), правого глаза: (415, 317)


In [44]:
add_glasses('ron_wesley.jpg', 'output_with_glasses.jpg', 'red', 5)

Количество найденных лиц: 1
Найдено лицо с координатами: x=220, y=115, ширина=183, высота=183
Количество найденных глаз: 2
Координаты левого глаза: (272, 184), правого глаза: (350, 190)


In [45]:
add_glasses('longbottom.jpg', 'output_with_glasses.jpg', 'red', 5)

Количество найденных лиц: 1
Найдено лицо с координатами: x=167, y=125, ширина=287, высота=287
Количество найденных глаз: 2
Координаты левого глаза: (260, 238), правого глаза: (353, 236)


In [46]:
add_glasses('emma-watson2.jpg', 'output_with_glasses.jpg', 'red', 5)

Количество найденных лиц: 1
Найдено лицо с координатами: x=89, y=105, ширина=223, высота=223
Количество найденных глаз: 2
Координаты левого глаза: (159, 195), правого глаза: (242, 186)
